## Find how many cases are related to frozen/live/scales (406 itslaw)

In [14]:
import pandas as pd
import re
df = pd.read_csv('fulltext - fulltext.csv')

def get_df_filter_by_keywords(keywords):
    _df = df[df['text'].str.contains(keywords, flags=re.IGNORECASE, regex=True)].reset_index().drop('index', axis=1)
    print('total cases: ', len(_df))
    return _df

def print_para_with_keywords(_df, keywords):
    case_count = 0
    for text in _df['text']:
        ps = [p for p in text.split('\n') if any(keyword in p for keyword in keywords.split('|'))]
        if len(ps) > 0:
            case_count = case_count +1
            print(ps)
            print('\n')
    print('Total cases: ', case_count)

In [11]:
df_scale = get_df_filter_by_keywords('鳞片|甲壳|甲片|炮山甲|甲皮')
df_live = get_df_filter_by_keywords('活体') #幼体 +0
df_dead = get_df_filter_by_keywords('冻体|冰冻|速冻|冻肉|肉块|死体')

total cases:  163
total cases:  171
total cases:  143


In [76]:
print('percentage:', int(163 /406 *100), int(171/406*100), int(143/406*100))

percentage: 40 42 35


In [ ]:
# print_para_with_keywords(df_scale, '鳞片')

# Geocode

In [62]:
import geocoder
import time
pd.options.mode.chained_assignment = None  # default='warn'

def get_df_lat_lng(_df):
    df_court = _df[['date', 'court']]
    success_count = 0
    for index, address in df_court['court'].iteritems():
        try:
            g = geocoder.baidu(address, key='CpEiE0gRf2ulr1YbkjpbPxhgolTuhX6l')
            if g.ok:
                df_court.loc[index, 'x'] = g.json['lat']
                df_court.loc[index, 'y'] = g.json['lng']
                df_court.loc[index, 'confidence'] = g.json['confidence']
                success_count = success_count + 1
            else :
                print(address, 'na')
        except:
            print('Exception')
    print(success_count)
    df_court.columns = ['year', 'court', 'lat', 'lng', 'confidence']
    return df_court

def to_csv(_df, name):
    _df.to_csv(name, encoding='utf_8_sig')
    
def reverse_code_province(_df):
    success_count = 0
    for index, row in _df.iterrows():
        try:
            g = geocoder.baidu([row.lat, row.lng], method='reverse', key='CpEiE0gRf2ulr1YbkjpbPxhgolTuhX6l')        
            if g.ok:
                _df.loc[index, 'province'] = g.json['raw']['addressComponent']['province'].replace('省', '')
                success_count = success_count + 1
            else :
                print(address, 'na')
        except:
            print('Exception')
    print (success_count)
    return _df

def count_per_province(_df):
    province_count = _df.groupby('province').count()
    province_count_json = list()
    province_count_total = 0
    for index, row in province_count.iterrows():
        province_count_total = province_count_total + row['year']
        province_count_json.append({'name': index, 'value':row['year']})
    print('total: ', province_count_total)
    return province_count_json

In [69]:
# df_scale_addr = get_df_lat_lng(df_scale)
# df_live_addr = get_df_lat_lng(df_live)
df_dead_addr = get_df_lat_lng(df_dead)

df_dead_addr.head()

143


,year,court,lat,lng,confidence
0,2016,勐海县人民法院,21.957359,100.445475,80.0
1,2018,广州市白云区人民法院,23.206978,113.270531,80.0
2,2018,钦州市钦南区人民法院,21.944469,108.657100,80.0
3,2017,江山市人民法院,28.756655,118.649222,80.0
4,2015,富阳市人民法院,30.065862,119.950352,80.0


In [70]:
# reverse_code_province(df_scale_addr)
# reverse_code_province(df_live_addr)
reverse_code_province(df_dead_addr)

143


,year,court,lat,lng,confidence,province
0,2016,勐海县人民法院,21.957359,100.445475,80.0,云南
1,2018,广州市白云区人民法院,23.206978,113.270531,80.0,广东
2,2018,钦州市钦南区人民法院,21.944469,108.657100,80.0,广西壮族自治区
3,2017,江山市人民法院,28.756655,118.649222,80.0,浙江
4,2015,富阳市人民法院,30.065862,119.950352,80.0,浙江
5,2014,南京市鼓楼区人民法院,32.071248,118.750943,80.0,江苏
6,2015,珠海市中级人民法院,22.285510,113.543629,80.0,广东
7,2016,芒市人民法院,24.445034,98.592952,80.0,云南
8,2018,石门县人民法院,29.586346,111.370523,80.0,湖南
9,2013,钦州市中级人民法院,21.945027,108.667099,75.0,广西壮族自治区


In [71]:
# to_csv(df_scale_addr, './scale-cases-all-addr.csv')
# to_csv(df_live_addr, './live-cases-all-addr.csv')
to_csv(df_dead_addr, './dead-cases-all-addr.csv')

In [63]:
count_per_province(df_scale_addr)

total:  163


[{'name': '上海市', 'value': 2},
 {'name': '云南', 'value': 78},
 {'name': '内蒙古自治区', 'value': 1},
 {'name': '北京市', 'value': 1},
 {'name': '吉林', 'value': 1},
 {'name': '四川', 'value': 3},
 {'name': '安徽', 'value': 6},
 {'name': '山东', 'value': 1},
 {'name': '广东', 'value': 19},
 {'name': '广西壮族自治区', 'value': 6},
 {'name': '江苏', 'value': 2},
 {'name': '江西', 'value': 2},
 {'name': '河北', 'value': 1},
 {'name': '河南', 'value': 1},
 {'name': '浙江', 'value': 22},
 {'name': '湖北', 'value': 1},
 {'name': '湖南', 'value': 5},
 {'name': '福建', 'value': 5},
 {'name': '西藏自治区', 'value': 2},
 {'name': '贵州', 'value': 1},
 {'name': '辽宁', 'value': 2},
 {'name': '重庆市', 'value': 1}]

In [68]:
count_per_province(df_live_addr)

total:  171


[{'name': '上海市', 'value': 1},
 {'name': '云南', 'value': 35},
 {'name': '吉林', 'value': 1},
 {'name': '广东', 'value': 33},
 {'name': '广西壮族自治区', 'value': 26},
 {'name': '江苏', 'value': 5},
 {'name': '河北', 'value': 1},
 {'name': '河南', 'value': 4},
 {'name': '浙江', 'value': 40},
 {'name': '海南', 'value': 2},
 {'name': '湖南', 'value': 7},
 {'name': '福建', 'value': 15},
 {'name': '贵州', 'value': 1}]

In [72]:
count_per_province(df_dead_addr)

total:  143


[{'name': '云南', 'value': 26},
 {'name': '吉林', 'value': 1},
 {'name': '广东', 'value': 28},
 {'name': '广西壮族自治区', 'value': 20},
 {'name': '江苏', 'value': 4},
 {'name': '江西', 'value': 1},
 {'name': '河北', 'value': 2},
 {'name': '河南', 'value': 2},
 {'name': '浙江', 'value': 37},
 {'name': '海南', 'value': 1},
 {'name': '湖北', 'value': 1},
 {'name': '湖南', 'value': 6},
 {'name': '福建', 'value': 13},
 {'name': '重庆市', 'value': 1}]

In [15]:
keyword = 'Manis|PHOLIDOTA|Manidae|中国穿山甲|爪哇穿山甲'
print_para_with_keywords(get_df_filter_by_keywords(keyword), keyword)

total cases:  94
['公诉机关指控，2015年期间，被告人熊某某分别在缅甸小勐拉、勐海县布朗山乡南很村寨收购疑似珍贵野生动物、珍贵野生动物制品共计60件。2015年11月28日8时许，被告人熊某某在勐海县勐混镇集市出售疑似珍贵野生动物、珍贵野生动物制品时，被勐海县民警查获。经鉴定，涉案60件疑似珍贵野生动物、珍贵野生动物制品中有2件亚洲象皮、36片中国穿山甲甲片、3件针尾绿鸠死体、1件苍鹰死体、1件白鹇死体、3件绿翅金鸠死体、1件小鸦鹃死体、13件棕腹柳莺死体。其中亚洲象为国家一级重点保护野生动物，中国穿山甲、针尾绿鸠、苍鹰、白鹇为国家二级重点保护野生动物，绿翅金鸠、小鸦鹃、棕腹柳莺被列入《国家保护的有益或有重要经济、科学研究价值的陆生野生动物名录》物种；上述珍贵野生动物、珍贵野生动物制品价值共计人民币8975.56元。', '经审理查明，2015年期间，被告人熊某某分别在缅甸小勐拉、勐海县布朗山乡南很村寨收购疑似珍贵野生动物、珍贵野生动物制品共计60件。2015年11月28日8时许，被告人熊某某在勐海县勐混镇集市出售疑似珍贵野生动物、珍贵野生动物制品时，被勐海县民警查获。经鉴定，涉案60件疑似珍贵野生动物、珍贵野生动物制品中有2件亚洲象皮、36片中国穿山甲甲片、3件针尾绿鸠死体、1件苍鹰死体、1件白鹇死体、3件绿翅金鸠死体、1件小鸦鹃死体、13件棕腹柳莺死体。其中亚洲象为国家一级重点保护野生动物，中国穿山甲、针尾绿鸠、苍鹰、白鹇为国家二级重点保护野生动物，绿翅金鸠、小鸦鹃、棕腹柳莺被列入《国家保护的有益或有重要经济、科学研究价值的陆生野生动物名录》物种；上述珍贵野生动物、珍贵野生动物制品价值共计人民币8975.56元。', '8、鉴定意见书、鉴定意见通知书，证实涉案动物及动物制品中，经鉴定亚洲象为国家一级重点保护野生动物（死体价值111.2元）、中国穿山甲（死体价值180.36元）针尾绿鸠（死体价值4008元）、苍鹰（死体价值3340元）白鹇（死体价值1335元）为国家二级重点保护野生动物，绿翅金鸠、小鸦鹃、棕腹柳莺被列入《国家保护的有益或有重要经济、科学研究价值的陆生野生动物名录》物种的情况，该鉴定意见已经告知被告人熊某某的情况。', '二、涉案赃物2件亚洲象皮、36片中国穿山甲甲片、3件针尾绿鸠死体、1件苍鹰死体、1件白鹇死体、3件绿翅金鸠死体